In [1]:
# importing modules
import numpy as np
import numpy.linalg as LA
from sklearn import svm
from scipy.spatial.distance import pdist, squareform
import scipy

In [2]:
f = open("/home/tandon/IIIT-H/3rd/SMAI/arcene/arcene_train.data")
X = np.zeros((100, 10000))
row = 0
col = 0
for line in f:
    for token in line.split():
        X[row, col] = int(token)
        col += 1
    col = 0
    row += 1
f.close()

In [3]:
# read the class labels associated with the training data
f = open("/home/tandon/IIIT-H/3rd/SMAI/arcene/arcene_train.labels")
labels = np.zeros(100)
i = 0
for line in f:
    digit = float(line.strip())
    labels[i] = digit
    i += 1
print(labels.shape)
f.close()

(100,)


In [4]:
labels[4]

-1.0

In [5]:
def rbf_value(x1, x2, gamma):
    diff = x1 - x2
    mod_diff = LA.norm(diff)**2
    return np.exp(-1 * gamma * mod_diff)

In [6]:
def project_point(point, X, alphas, gamma):
    kvec = np.array([np.sum((point - row)**2) for row in X])
    kvec = np.exp(- gamma * kvec)
    return alphas.T.dot(kvec)
    print('Projected point')

In [7]:
# finding the normalized eigen vectors associated with the training data
components = 1
gamma = 1/(np.var(X)*50)
alphas, lambdas = kernel_pca(X, gamma, components)

NameError: name 'kernel_pca' is not defined

In [8]:
def create_M(X, category):
    gamma = 1/(np.var(X)*50)
    num_samples = X.shape[0]
    num_category = category.shape[0]
    M = np.zeros(num_samples)
    for i in range(num_samples):
        for j in range(num_category):
            M[i] += rbf_value(X[i], category[j], gamma)
        M[i] = M[i]/num_category
    return M

def create_K(X, category):
    gamma = 1/(np.var(X)*50)
    num_samples = X.shape[0]
    num_category = category.shape[0]
    K = np.zeros((num_samples, num_category))
    for i in range(num_samples):
        for j in range(num_category):
            K[i, j] = rbf_value(X[i], category[j], gamma)
    return K

In [9]:
num_samples = X.shape[0]
bool_idx = (labels > 0)
positive_class = X[bool_idx, :]
num_positive = positive_class.shape[0]
print('X shape', X.shape)
print('positive_class shape', num_positive)

bool_idx = (labels < 0)
negative_class = X[bool_idx, :]
num_negative = negative_class.shape[0]
print('negative_class shape', num_negative)

M_positive = create_M(X, positive_class)
M_negative = create_M(X, negative_class)


M_diff = M_negative - M_positive
print('M_diff shape', M_diff.shape)

K_positive = create_K(X, positive_class)
K_negative = create_K(X, negative_class)

print('K_positive shape', K_positive.shape)
print('K_negative shape', K_negative.shape)

interim_positive = np.eye(num_positive) - np.ones((num_positive, num_positive))*(1/num_positive)
interim_negative = np.eye(num_negative) - np.ones((num_negative, num_negative))*(1/num_negative)

print('interim_positive shape', interim_positive.shape)
print('interim_negative shape', interim_negative.shape)

N_positive = K_positive.dot(interim_positive)
N_positive = N_positive.dot(K_positive.T)

N_negative = K_negative.dot(interim_negative)
N_negative = N_negative.dot(K_negative.T)

print('N_positive shape', N_positive.shape)
print('N_negative shape', N_negative.shape)

N = N_positive + N_negative

N_inv = LA.inv(N)
print('N_inv shape', N_inv.shape)

alpha = N_inv.dot(M_diff)
print('alpha shape', alpha.shape)


X shape (100, 10000)
positive_class shape 44
negative_class shape 56
M_diff shape (100,)
K_positive shape (100, 44)
K_negative shape (100, 56)
interim_positive shape (44, 44)
interim_negative shape (56, 56)
N_positive shape (100, 100)
N_negative shape (100, 100)
N_inv shape (100, 100)
alpha shape (100,)


In [10]:
def project_point(point, X, alphas, gamma):
    kvec = np.array([np.sum((point - row)**2) for row in X])
    kvec = np.exp(- gamma * kvec)
    return alphas.T.dot(kvec)

In [11]:
Y = np.zeros(num_samples)
gamma = 1/(np.var(X)*50)
clf = svm.SVC()
for i in range(num_samples):
    Y[i] = project_point(X[i], X, alpha, gamma)

Y = np.reshape(Y, (num_samples, 1))
print('Y shape', Y.shape)
print('labels shape', labels.shape)

clf.fit(Y, labels)

Y shape (100, 1)
labels shape (100,)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:
# read the validation data
f = open("/home/tandon/IIIT-H/3rd/SMAI/arcene/arcene_valid.data")
X_valid = np.zeros((100, 10000))
row = 0
col = 0
for line in f:
    for token in line.split():
        X_valid[row, col] = int(token)
        col += 1
    col = 0
    row += 1
f.close()
num_samples = X_valid.shape[0]

In [13]:
# read the class labels associated with the validation data
f = open("/home/tandon/IIIT-H/3rd/SMAI/arcene/arcene_valid.labels")
labels_valid = np.zeros(100)
i = 0
for line in f:
    digit = float(line.strip())
    labels_valid[i] = digit
    i += 1
f.close()

In [14]:
# validating our training 
correct = 0
gamma = 1/(np.var(X)*50)
for i in range(num_samples):
    projection = project_point(X_valid[i], X_valid, alpha, gamma)
#     print('Projection', projection)
    if clf.predict(projection)[0] > 0:
        if labels_valid[i] > 0:
            correct += 1
    else:
        if labels_valid[i] < 0:
            correct += 1
print('Correct Efficiency: ', correct/100)
            
    
    

Correct Efficiency:  0.46


### Observation

For arcene it gave 0.46

For dorothea it gave 0.85
